In [1]:
!pip install datasets soundfile librosa jiwer pytorch-lightning transformers[torch] evaluate -q


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import torch
import transformers
from datasets import load_dataset, load_metric, Audio
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2Config,
    Wav2Vec2ConformerForCTC,
    Wav2Vec2Processor,
    TrainingArguments,
    Trainer,
    AutoProcessor,
    EarlyStoppingCallback#,
    #ModelCheckpoint
    
)
import pandas as pd
import datasets
import re
import json
from datasets import DatasetDict
from datasets import Dataset
from dataclasses import dataclass, field
import evaluate
from typing import Dict, List, Optional, Union

#from pytorch_lightning import Trainer
# pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.callbacks import ModelCheckpoint
#from transformers import IntervalStrategy,ModelCheckpoint
#from keras.callbacks import ModelCheckpoint

from transformers import IntervalStrategy
from transformers.models.wav2vec2_conformer import Wav2Vec2ConformerModel
from torch.nn.parallel import DistributedDataParallel

In [164]:
loaded_dataset = load_dataset("lj_speech",split="train")

Found cached dataset lj_speech (/home/ubuntu/.cache/huggingface/datasets/lj_speech/main/1.1.0/6e03b5308ab68705d902e2bbeaff59feb283307770fafb8d6704e604b6d179aa)


In [165]:
conversion_dict = {"train":loaded_dataset,"val":loaded_dataset}
dataset = DatasetDict(conversion_dict)

In [167]:
dataset = dataset.remove_columns(["text","id","file"])
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [168]:
chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"[\]()èüâéàê”“’]'
def preprocess(batch):
    batch["audio"] = dataset["train"][0]["audio"]["array"]
    batch["sampling_rate"] = dataset["train"][0]["audio"]["sampling_rate"]
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["normalized_text"]).lower()
    return batch

In [169]:
dataset = dataset.map(preprocess)
dataset = dataset.remove_columns(["normalized_text"])

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/lj_speech/main/1.1.0/6e03b5308ab68705d902e2bbeaff59feb283307770fafb8d6704e604b6d179aa/cache-4c4d3266bca241fe.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/lj_speech/main/1.1.0/6e03b5308ab68705d902e2bbeaff59feb283307770fafb8d6704e604b6d179aa/cache-4c4d3266bca241fe.arrow


In [176]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-conformer-rope-large-960h-ft")

In [178]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["audio"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [179]:
data_prepared = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/13100 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/lj_speech/main/1.1.0/6e03b5308ab68705d902e2bbeaff59feb283307770fafb8d6704e604b6d179aa/cache-15ccb404d5ced137_*_of_00004.arrow


In [180]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [181]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [182]:
wer_metric = evaluate.load("wer")

In [183]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [184]:
model = Wav2Vec2ConformerForCTC.from_pretrained(
    "facebook/wav2vec2-conformer-rope-large-960h-ft", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [185]:
for name, param in model.wav2vec2_conformer.feature_extractor.named_parameters():
    param.requires_grad = False

conv_layers.0.conv.weight
conv_layers.0.conv.bias
conv_layers.0.layer_norm.weight
conv_layers.0.layer_norm.bias
conv_layers.1.conv.weight
conv_layers.1.conv.bias
conv_layers.1.layer_norm.weight
conv_layers.1.layer_norm.bias
conv_layers.2.conv.weight
conv_layers.2.conv.bias
conv_layers.2.layer_norm.weight
conv_layers.2.layer_norm.bias
conv_layers.3.conv.weight
conv_layers.3.conv.bias
conv_layers.3.layer_norm.weight
conv_layers.3.layer_norm.bias
conv_layers.4.conv.weight
conv_layers.4.conv.bias
conv_layers.4.layer_norm.weight
conv_layers.4.layer_norm.bias
conv_layers.5.conv.weight
conv_layers.5.conv.bias
conv_layers.5.layer_norm.weight
conv_layers.5.layer_norm.bias
conv_layers.6.conv.weight
conv_layers.6.conv.bias
conv_layers.6.layer_norm.weight
conv_layers.6.layer_norm.bias


In [186]:
training_args = TrainingArguments(
    output_dir="./wav2vec2_conformer",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=500,
    group_by_length=True,
    fp16=True
)

In [187]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["val"],
    tokenizer=processor.feature_extractor,
)

In [188]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 